In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import numpy as np
import seaborn as sns
import time
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sb
###

# Plot settings
%config InlineBackend.figure_format = 'svg'
sb.set()
from tqdm import tqdm

import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm

import sys
sys.path.insert(0, '/directory/tothe/handshakefile/')
sys.path.append('/home/ak/Documents/PaperCode/stylised_facts')
import stylised_facts_data_utilities as sfd_utils
import lob_for_futures as lobfut
%matplotlib inline
import os
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from scipy.stats import norm, ttest_ind

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
import logging


def logger(name, level=logging.INFO):
    tmp_logger = logging.getLogger(name)
    if not tmp_logger.handlers:
        formatter = logging.Formatter('%(asctime)s %(name)s  %(levelname)s %(message)s')
        handler = logging.StreamHandler()
        handler.setFormatter(formatter)

        tmp_logger.setLevel(level)
        tmp_logger.addHandler(handler)
    return tmp_logger

In [3]:
## data files
laptop_OS_folder = '/media/ak/OS/Data/FuturesDataSemiProcessed'
LaCie_ProcessedData = '/media/ak/LaCie/ProcessedSampledData/'
#returns_data = '/media/ak/T7/August11th2022Experiments/Returns/'
t7 = '/media/ak/T7/'
june_ext = os.path.join(t7, 'June4th2022Experiments')
returns_data = [f for f in os.listdir(june_ext) if '_returns' in f]
exp_destination = '/media/ak/T7/August11th2022Experiments/'

In [4]:
# here i start with RX1 to do all the experiments in one go
symbols  = os.listdir(laptop_OS_folder)
symbol_test_folder  = os.path.join(laptop_OS_folder, symbols[0])
rx_folder = os.path.join(laptop_OS_folder, 'RX1')
files = os.listdir(rx_folder)

In [11]:
bar_keys = ['tick', 'volume', 'calendar', 'dollar']
files_idx = 5
date_idx = files[files_idx].split(".")[0]
idx_file = os.path.join(rx_folder, files[files_idx])
tick_df = pd.read_pickle(idx_file)[date_idx]['tick']
volume_df = pd.read_pickle(idx_file)[date_idx]['volume']
calendar_df = pd.read_pickle(idx_file)[date_idx]['calendar']
dollar_df = pd.read_pickle(idx_file)[date_idx]['dollar']

In [12]:
# need to make a choice here of what variables I am going to keep and where

In [13]:
tick_df_micro = lobfut.apply_micro_structure_features(tick_df) # get micro structure df
vol_class = lobfut.volatilityEstimation(tick_df) # get the vol class

Applying micro-structure features


In [14]:
# features I need: micro_price changes:
tick_experiment_df = tick_df_micro[['micro_price', 'price_imbalance',
                                    'pct_change_micro_price', 'weighted_activity_spread',
                                    'total_traded_volume_open',
                                    'total_traded_volume_high', 'total_traded_volume_low',
                                    'total_traded_volume_close']]



In [38]:
x =tick_experiment_df.loc[: , ['pct_change_micro_price']].fillna(0) - tick_experiment_df.loc[:,
                                                                   ['pct_change_micro_price']].fillna(0).mean() / tick_experiment_df.loc[
                                                                                                                  :, [
                                                                                                                         'pct_change_micro_price']].fillna(
    0).fillna(0).std()
tick_experiment_df['normalised_returns'] = x

/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
tick_experiment_df['Returns_MinMax'] = np.array(
    scaler.fit_transform(tick_experiment_df.pct_change_micro_price.values.reshape(-1, 1))).reshape(
    tick_experiment_df.pct_change_micro_price.values.shape[0], )
rs, rk = vol_class.realised_skewness_kurtosis()

tick_experiment_df['skew'] = pd.Series(rs.reshape(rs.shape[0],)).replace([np.inf, -np.inf], 0).fillna(0)
tick_experiment_df['kurt'] = pd.Series(rk.reshape(rk.shape[0],)).replace([np.inf, -np.inf], 0).fillna(0)
tick_experiment_df['total_traded_volume_median'] = tick_experiment_df.loc[:,['volume' in i for i in tick_experiment_df.columns]].mean(axis=1)

/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


AttributeError: 'Series' object has no attribute 'reshape'

In [40]:
## vol features

tick_experiment_df['GK_vol'] = pd.Series(list(vol_class.garmanKlass(5)))
tick_experiment_df['GK_vol'] =tick_experiment_df['GK_vol'].fillna(tick_experiment_df['GK_vol'].median())
tick_experiment_df['arrival_rates'] = pd.Series(vol_class.arrival_rates().reshape(vol_class.arrival_rates().shape[0],)).replace([np.inf, -np.inf], 0).fillna(0)



/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A va

In [ ]:


tick_experiment_df.drop(['total_traded_volume_open',
                         'total_traded_volume_high', 'total_traded_volume_low',
                         'total_traded_volume_close'], axis=1)
tick_experiment_df =tick_experiment_df.fillna(0)


In [ ]:
#tripower_quarticity
tick_experiment_df.columns.values

In [ ]:
tick_experiment_df.head(5).

In [ ]:
tick_experiment_df.loc[:,['volume' in i for i in tick_experiment_df.columns]].mean(axis=1)